<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Python-Notebook-Banners/Code_challenge.png"  style="display: block; margin-left: auto; margin-right: auto;";/>
</div>

# Code challenge: Random forest regression
© ExploreAI Academy

In this code challenge, we'll test our knowledge of how to create an ensemble model known as a random forest. We will train this new model using the world population data. 

⚠️ **Note that this code challenge is graded and will contribute to your overall marks for this module. Submit this notebook for grading. Note that the names of the functions are different in this notebook. Transfer the code in your notebook to this submission notebook**

### Instructions

- **Do not add or remove cells in this notebook. Do not edit or remove the `### START FUNCTION` or `### END FUNCTION` comments. Do not add any code outside of the functions you are required to edit. Doing any of this will lead to a mark of 0%!**

- Answer the questions according to the specifications provided.

- Use the given cell in each question to see if your function matches the expected outputs.

- Do not hard-code answers to the questions.

- The use of StackOverflow, Google, and other online tools is permitted. However, copying a fellow student's code is not permissible and is considered a breach of the Honour code. Doing this will result in a mark of 0%.

### Imports

In [ ]:
import numpy as np
import pandas as pd
from numpy import array
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [ ]:
population_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/AnalyseProject/world_population.csv', index_col='Country Code')
meta_df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/AnalyseProject/metadata.csv', index_col='Country Code')

In [ ]:
population_df.head()

In [ ]:
meta_df.head()

### Question 1

The world population data spans from 1960 to 2017. We'd like to build a predictive model that can give us the best guess at what the world population in a given year was. However, we want to compute this estimate for only _countries within a given income group_. 

First, however, we need to organise our data such that the sklearn's `RandomForestRegressor` class can train on our data. To do this, we will write a function that takes as input an income group and returns a 2-d numpy array that contains the year and the measured population.

_**Function Specifications:**_
* Should take a `str` argument, called `income_group_name` as input and return a numpy `array` type as output.
* Set the default argument of `income_group_name` to equal `'Low income'`.
* If the specified value of `income_group_name` does not exist, the function must raise a `ValueError`.
* The array should only have two columns containing the year and the population, in other words, it should have a shape `(?, 2)` where `?` is the length of the data.
* The values within the array should be of type `np.int64`. 

_**Further Reading:**_

Data types are associated with memory allocation. As such, your choice of data type affects the precision of computations in your program. For example, the `np.int` data type in numpy can only store values between -2147483648 to 2147483647 and assigning values outside this range for variables of this data type may cause run-time errors. To avoid this, we can use data types with larger memory capacity e.g. `np.int64`.

https://docs.scipy.org/doc/numpy/user/basics.types.html

In [ ]:
### START FUNCTION
def get_total_pop_by_income(income_group_name='Low income'):
    
    comb_df = population_df.copy().join(meta_df)
    if income_group_name not in comb_df['Income Group'].unique():
        raise ValueError('Specified income group: "{}", not found in DataFrame'.format(income_group_name))
    comb_df = comb_df[comb_df['Income Group'] == income_group_name].drop(['Region', 'Income Group', 'Special Notes'], axis=1)
    comb_df = comb_df.sum()
    comb_df = comb_df.astype(np.int64)
    X = list(map(np.int64, population_df.columns))
    Y = np.array(comb_df)
    return np.vstack([X,Y]).T
### END FUNCTION

In [ ]:
data = get_total_pop_by_income('High income')

_**Expected Outputs:**_
```python
get_total_pop_by_income('High income')
```
> ```
array([[      1960,  769889923],
       [      1961,  781225329],
       [      1962,  791207437],
       [      1963,  801108277],
       ...
       [      2015, 1211252041],
       [      2016, 1218629612],
       [      2017, 1225514228]])
```




### Question 2

Now that we have have our data, we need to split this into a set of variables we will be training on, and the set of variables that we will make our predictions on.

Sklearn has a bunch of built-in functionality for creating training and testing sets. Our task is to implement a k-fold cross validation split of the data using sklearn's `KFold` [class](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) (which has already been imported into this notebook for your convenience). 

Using this knowledge, write a function which uses sklearn's `KFold` [class](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) internally, and that will take as input a 2-d numpy array and an integer `K` corresponding to the number of splits. This function will then return a list of tuples of length `K`. Each tuple in this list should consist of a `train_indices` list and a `test_indices` list containing the training/testing data point indices for that particular $K^{th}$ split.

_**Function Specifications:**_
* Should take a 2-d numpy `array` and an integer `K` as input.
* Should use sklearn's `KFold` [class](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html).
* Should return a list of `K` `tuples` containing a list of training and testing indices corresponding to the data points that belong to a particular split. For example, given an array called `data` and an integer `K`, the function should return: 
>```
data_indices = [(list_of_train_indices_for_split_1, list_of_test_indices_for_split_1)
                  (list_of_train_indices_for_split_2, list_of_test_indices_for_split_2)
                  (list_of_train_indices_for_split_3, list_of_test_indices_for_split_3)
                                                   ...
                                                   ...
                  (list_of_train_indices_for_split_K, list_of_test_indices_for_split_K)]
```

* The `shuffle` argument in the KFold object should be set to `False`.

**_Hint_**: To see an example of how to use the `KFold` object enter `help(KFold)` in a new notebook cell

In [ ]:
### START FUNCTION
def sklearn_kfold_split(data,K):
    kfold = KFold(n_splits=K)
    data_indices = [(train_idx_list,test_idx_list) for train_idx_list,test_idx_list in kfold.split(data)]
    return list(data_indices)
### END FUNCTION

In [ ]:
data = get_total_pop_by_income('High income');
sklearn_kfold_split(data,4)

_**Expected Outputs:**_
```python
data = get_total_pop_by_income('High income')
sklearn_kfold_split(data,4)
```
> ```
[(array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
         32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
         49, 50, 51, 52, 53, 54, 55, 56, 57]),
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 30, 31,
         32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
         49, 50, 51, 52, 53, 54, 55, 56, 57]),
  array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 44, 45, 46, 47,
         48, 49, 50, 51, 52, 53, 54, 55, 56, 57]),
  array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43]),
  array([44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]))]
 ```

### Question 3

Now that we have formatted our data, we can fit a model using sklearn's `RandomForestRegressor` class. We'll write a function that will take as input the data indices (consisting of the train and test indices for each split) that we created in the last question, train a different `RandomForestRegressor` on each split and return the model that obtains the best testing set performance across all K splits.

**Important Note:** Due to the random initialisation process used within sklearn's `RandomForestRegressor` class, you will need to fix the value of the `random_state` argument in order to get repeatable and predictable results.

_**Function Specifications:**_
* Should take a 2-d numpy array (i.e. the data) and `data_indices` (a list of `(train_indices,test_indices)` tuples) as input.
* For each `(train_indices,test_indices)` tuple in `data_indices` the function should:
    * Train a new `RandomForestRegressor` model on the portion of data indexed by `train_indices`
    * Evaluate the trained `RandomForestRegressor` model on the portion of data indexed by `test_indices` using the **mean squared error** (which has also been imported for your convenience).
* After training and evaluating the `RandomForestRegressor` models, the function should return the `RandomForestRegressor` model that obtained the highest testing set `mean_square_error` over its allocated data split across all trained models. 
* The trained `RandomForestRegressor` models should be trained with `random_state` equal `42`, all other parameters should be left as default.

**_Hint_**: for each tuple in the `data_indices` list, you can obtain `X_train`,`X_test`, `y_train`, `y_test` as follows:  
>```
    X_train, y_train = data[train_indices,0],data[train_indices,1]
    X_test, y_test = data[test_indices,0],data[test_indices,1]
```



In [ ]:
### START FUNCTION
def best_k_model(data,data_indices):
    model_list = []
    model_mse = []
    for train_indices,test_indices in data_indices:
        
        # init model
        model = RandomForestRegressor(random_state=42)
        X_train, y_train = data[train_indices,0],data[train_indices,1]
        X_test, y_test = data[test_indices,0],data[test_indices,1]
        
        # train model
        model.fit(X_train[:,np.newaxis], y_train) 
        model_list.append(model)
        
        # test model
        mse = mean_squared_error(model.predict(X_test[:,np.newaxis]),y_test)
        model_mse.append(mse)
    
    # get index of highest mse
    best_model_idx = np.argmax(model_mse)
    
    return model_list[best_model_idx]
### END FUNCTION

In [ ]:
data = get_total_pop_by_income('High income')
data_indices = sklearn_kfold_split(data,5)

best_model = best_k_model(data,data_indices)
best_model.predict([[1960]])

_**Expected Outputs:**_
```python
best_model.predict([[1960]]) == array([8.85170916e+08])
```

#  

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>